Requirements: Install ipywidgets 

# Datasets Ensenyament febrer 2023

En el marc de la [Setmana del Govern Obert](https://educacio.gencat.cat/ca/departament/transparencia/govern-obert/) es varen publicar dos datasets interessants, ["Plantilles del personal docent dels centres i serveis educatius i forma d'ocupació"](https://analisi.transparenciacatalunya.cat/Educaci-/Plantilles-del-personal-docent-dels-centres-i-serv/9zqj-j7ei) i "Concerts educatius: Unitat, alumnes, dotació de plantilla i despesa".

Aquest bloc conté unes visualitzacions sobre les plantilles de profes d'informàtica de Girona basades en les dades del primer dels conjunts de dades esmentats.

L'objectiu d'aquest bloc és tenir una foto de la situació actual del professorat d'informàtica a les comarques gironines així com a servir d'inspiració per a altres visualitzacions i usus que se li vulguin donar a aquestes dades o altres dades obertes. 

Aquest projecte s'ha iniciat a partir del repositori  [del repo a GitHub](https://github.com/ctrl-alt-d/dataset_ensenyament_2022).

## Càrrega de les dades

Carreguem el dataset de plantilles i ens quedem amb els informàtics gironins.

In [12]:
import pandas as pd

url="https://analisi.transparenciacatalunya.cat/api/views/9zqj-j7ei/rows.csv?accessType=DOWNLOAD&sorting=true"

df = pd.read_csv(url, on_bad_lines='skip')

# El codi centre és un string:
df.CODI_CENTRE= df.CODI_CENTRE.astype(str)

# Ens quedem amb els informàtics gironins:
df_girona = df.loc[df.COD_LLOC.isin( ['507','627'] ) & df.CODI_CENTRE.str.startswith('8046748') ]

df_girona627 = df.loc[df.COD_LLOC.isin( ['627'] ) & df.CODI_CENTRE.str.startswith('08046748') ]
df_girona507 = df.loc[df.COD_LLOC.isin( ['507'] ) & df.CODI_CENTRE.str.startswith('08046748') ]

In [13]:
df_girona.describe()

,CODI_DELEGACIÓ,TOT_DOT,CS,DD,IN,NF,PP,PS,PG,RF
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Dotació curs 2022-2023

Dotació per centre de professorat de les especialitats d'informàtica

In [9]:
import altair as alt

data = df_girona[df_girona.Curs=="2022-2023"].groupby(["NOM_MUNICIPI", "COD_LLOC"])["TOT_DOT"].sum().reset_index()

bars = alt.Chart().mark_bar().encode(
    alt.X("sum(TOT_DOT)",title="Dotació", stack='zero'),
    alt.Y("NOM_MUNICIPI:N", title=""),
    alt.Color("COD_LLOC", title="Especialitat"),
)

text_detall = alt.Chart().mark_text(dx=-13, color='white').encode(
    alt.X("sum(TOT_DOT)", stack='zero'),
    alt.Y("NOM_MUNICIPI:N", title=""),
    detail='COD_LLOC:N',
    text=alt.Text('sum(TOT_DOT):Q')
)

alt.layer(
        bars, text_detall, data=data
    ).interactive()

c:\Python311\Lib\site-packages\altair\utils\core.py:228: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  warnings.warn(


alt.LayerChart(...)

## Evolució de la Plantilla per centre

Nombre de professors per especialitat a cada centre.

In [11]:
import altair as alt

data = df_girona.groupby(["Curs", "NOM_MUNICIPI", "COD_LLOC"])["TOT_DOT"].sum().reset_index()

base = alt.Chart(data).encode(
    alt.Y("sum(TOT_DOT)",title="Dotació", stack='zero'),
    alt.X("Curs:N", title=""),
)

bars = base.mark_bar().encode(
    alt.Color("COD_LLOC", title="Especialitat"),
)

text_detall = base.mark_text(dy=10, color='white').encode(
    detail='COD_LLOC:N',
    text=alt.Text('sum(TOT_DOT):Q')
)

alt.layer(
        bars, text_detall
    ).facet(
        facet=alt.Column('NOM_MUNICIPI', title=""),
        columns=5
    ).interactive()


c:\Python311\Lib\site-packages\altair\utils\core.py:228: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  warnings.warn(


alt.FacetChart(...)

## Destinació del professorat als centres

Destinació del professorat als centres.

* Funcionari amb destinació definitiva al centre:
  * DD: Destinació definitiva
* Funcionari sense destinació definitiva al centre:
  * CS: Comissió de serveis
  * PP: funcionari amb destinació provisional
  * PS: funcionari que ha perdut la definitiva per supressió de la plaça al centre on estava i està provisionalment a aquesta destinació
* Interinatge, personal de reforç i no fix:
  * IN:	Interinatge
  * NF:	no fix
  * PG: personal reforç pel tema del COVID
  * RF: Valor de reforços

### Destinació curs 2022-2023

Destinació del professorat als centres curs 2022-2023 a les comarques gironines.

In [5]:
import altair as alt

# Tipus i colors de les destinacions
destinacions = ["DD", "CS", "PP", "PS", "IN", "NF", "PG", "RF"]
colors = ['#229954', '#2E86C1', '#2471A3', '#884EA0', '#D35400', '#CA6F1E', '#D68910', '#F39C12', '#D4AC0D']

# Taula curs, municipi, destinació, Dotació
data = pd.melt(df_girona[df_girona.Curs=="2021-2022"], id_vars=[], value_vars=destinacions, var_name="Destinació", value_name="Dotació").dropna()

# Camp addicional per a ordenació manual
data['order'] = data['Destinació'].replace(
    {val: i for i, val in enumerate(destinacions)}
)

base = alt.Chart(data).encode(
    theta=alt.Theta("sum(Dotació):Q", stack=True),
    radius=alt.Radius("sum(Dotació)", scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color=alt.Color("Destinació", title="Vinculació al centre", scale=alt.Scale(domain=destinacions, range=colors)),
)

c1 = base.mark_arc(innerRadius=20, stroke="#fff")

c2 = base.mark_text(radiusOffset=10).encode(text="sum(Dotació):Q")

alt.layer(c1, c2)

alt.LayerChart(...)

Tipus de plantilla per centre

In [11]:
import altair as alt

# Taula curs, municipi, destinació, Dotació
data = pd.melt(df_girona, id_vars=["Curs", "NOM_MUNICIPI"], value_vars=destinacions, var_name="Destinació", value_name="Dotació").dropna()

# Camp addicional per a ordenació manual
data['order'] = data['Destinació'].replace(
    {val: i for i, val in enumerate(destinacions)}
)

base = base = alt.Chart(data).encode(
    alt.Y("sum(Dotació)",title="Dotació", stack='zero' ),
    alt.X("Curs:N", title=""),
    alt.Order("order", sort="descending" ),
)

bars = base.mark_bar().encode(
    alt.Color("Destinació", title="Vinculació al centre", scale=alt.Scale(domain=destinacions, range=colors)),
)

text_detall = base.mark_text(dy=10, color='white').encode(
    detail='Destinació:N',
    text=alt.Text('sum(Dotació):Q'),
)

alt.layer(
        bars, text_detall
    ).facet(
        facet=alt.Column('NOM_MUNICIPI', title=""),
        columns=5
    ).interactive()

alt.FacetChart(...)

### Tipus de destinació dels 507

In [57]:



import altair as alt

# Taula curs, municipi, destinació, Dotació
data = pd.melt(df_girona507, id_vars=["Curs", "NOM_MUNICIPI"], value_vars=destinacions, var_name="Destinació", value_name="Dotació").dropna()
print(data)
# Camp addicional per a ordenació manual
data['order'] = data['Destinació'].replace(
    {val: i for i, val in enumerate(destinacions)}
)

base = base = alt.Chart(data).encode(
    alt.Y("sum(Dotació)",title="Dotació", stack='zero' ),
    alt.X("Curs:N", title=""),
    alt.Order("order", sort="descending" ),
)

bars = base.mark_bar().encode(
    alt.Color("Destinació", title="Vinculació al centre", scale=alt.Scale(domain=destinacions, range=colors)),
)

text_detall = base.mark_text(dy=10, color='white').encode(
    detail='Destinació:N',
    text=alt.Text('sum(Dotació):Q'),
)

alt.layer(
        bars, text_detall
    ).facet(
        facet=alt.Column('NOM_MUNICIPI', title=""),
        columns=5
    ).interactive()

          Curs NOM_MUNICIPI Destinació  Dotació
0    2015-2016       Blanes         DD      3.0
2    2015-2016       Girona         DD      5.0
3    2015-2016         Salt         DD      1.0
5    2015-2016      Palamós         DD      1.0
6    2015-2016     Figueres         DD      4.0
..         ...          ...        ...      ...
549  2022-2023         Olot         IN      4.0
551  2022-2023      Palamós         IN      4.0
552  2022-2023         Salt         IN      2.0
553  2022-2023     Vidreres         IN      1.0
750  2021-2022         Salt         PG      0.5

[208 rows x 4 columns]


alt.FacetChart(...)

### Tipus de destinació dels 627

In [8]:
import altair as alt

# Taula curs, municipi, destinació, Dotació
data = pd.melt(df_girona627, id_vars=["Curs", "NOM_MUNICIPI"], value_vars=destinacions, var_name="Destinació", value_name="Dotació").dropna()

# Camp addicional per a ordenació manual
data['order'] = data['Destinació'].replace(
    {val: i for i, val in enumerate(destinacions)}
)

base = base = alt.Chart(data).encode(
    alt.Y("sum(Dotació)",title="Dotació", stack='zero' ),
    alt.X("Curs:N", title=""),
    alt.Order("order", sort="descending" ),
)

bars = base.mark_bar().encode(
    alt.Color("Destinació", title="Vinculació al centre", scale=alt.Scale(domain=destinacions, range=colors)),
)

text_detall = base.mark_text(dy=10, color='white').encode(
    detail='Destinació:N',
    text=alt.Text('sum(Dotació):Q'),
)

alt.layer(
        bars, text_detall
    ).facet(
        facet=alt.Column('NOM_MUNICIPI', title=""),
        columns=5
    ).interactive()

alt.FacetChart(...)